In [10]:
import feedparser as fp
import newspaper
from newspaper import Article
import time
from time import mktime
from datetime import datetime
from datetime import date
import pandas as pd
import json
import pprint
import dateutil

#### 1 Website data ####

## 1A ##  From JSON file - for final version

#with open('NewsPapers.json') as data_file: #Loads the JSON files with news URLs
#    companies = json.load(data_file)

## 1B ## From variable - this is for testing, makes it way faster
website = {"cnn": {"link": "http://edition.cnn.com/", "rss": "http://rss.cnn.com/rss/cnn_topstories.rss"},
          "cnbc":{"link": "https://www.cnbc.com/", "rss": "https://www.cnbc.com/id/10000664/device/rss/rss.html"},
          "bbc": {"rss": "http://feeds.bbci.co.uk/news/rss.xml","link": "http://www.bbc.com/"},
          "theguardian": {"rss": "https://www.theguardian.com/uk/rss", "link": "https://www.theguardian.com/international"},
          "washingtonpost": {"rss": "http://feeds.washingtonpost.com/rss/world", "link": "https://www.washingtonpost.com/"},
          "reuters": {"rss": "http://feeds.reuters.com/reuters/businessNews"},
          "fortune": {"rss": "https://fortune.com/feed"}}


#### 2 Todays date - for filtering the articles ####
today = str(date.today()) 
print("Today's date:", today)


#### 3 Scraping the news articles ####

text_list = []
source_list = []
article_list = []
date_list = []
time_list = []
title_list = []

for source, value in website.items():
    if 'rss' in value:
        d = fp.parse(value['rss']) #if there is an RSS value for a company in the website data, it will be extracted into d
        article={}
        
        for entry in d.entries:
            if hasattr(entry, 'published'):
                article['source'] = source
                source_list.append(article['source'])
                
                #getting the article URLs
                article['link'] = entry.link
                article_list.append(article['link'])
                
                #getting the article published dates
                date = (getattr(entry, 'published'))
                date = dateutil.parser.parse(date)
                date_formated = date.strftime("%Y-%m-%d")
                time_formated = date.strftime("%H:%M:%S %Z") #hour, minute, timezone (converted)
                date_list.append(date_formated)
                time_list.append(time_formated)
                
                #getting the titles
                content = Article(entry.link)
                try:
                    content.download() #downloading article content
                    #downloading takes approx. 3min to load
                    content.parse()                    
                except Exception as e: 
                    #in case the download fails, it prints the error and immediatly continues with downloading the next article
                    print(e)
                    print("continuing...")
                title = content.title #extract article titles
                title_list.append(title)
                text = content.text
                text_list.append(text)
                
#creating dicts for formatting and inserting to pandas df
source_dict = {'source':source_list}
link_dict = {'link':article_list}
date_dict = {'published_date':date_list}
time_dict = {'published_time':time_list}
title_dict = {'title':title_list}

#creating separate pandas dfs for each feature
source_df = pd.DataFrame(source_dict)
link_df = pd.DataFrame(link_dict)
date_df = pd.DataFrame(date_dict)
time_df = pd.DataFrame(time_dict)
title_df = pd.DataFrame(title_dict)

#join all pandas dfs together
news_df = source_df.join(link_df)
news_df = news_df.join(date_df)
news_df = news_df.join(time_df)
news_df = news_df.join(title_df)

#after 4,5 min, pandas DF sould be created with link, published_date, published_time, title and text
#4.5min
#with text 4min


Today's date: 2019-10-25


In [15]:
news_df_daily = news_df[news_df.published_date == today]

cnbc_test = news_df_daily[news_df_daily.source == "cnbc"]

#cnbc_test

news_df

#print(news_df.loc[[70]])

#news_df_daily

print(text_list)

['(CNN) Sen. Lindsey Graham and other congressional Republicans have slammed House Democrats for conducting their impeachment inquiry with depositions behind closed doors, but Republicans also used closed-door depositions during their impeachment inquiry of former President Bill Clinton two decades ago.\n\nThen-Rep. Lindsey Graham, at a November 1998 news conference one day after Special Prosecutor Ken Starr publicly testified before the House Judiciary Committee, praised the Judiciary panel\'s plans to hold depositions before conducting public hearings. Graham was a key Republican on the Judiciary Committee and was one of the House impeachment managers during the Senate trial that followed.\n\nAsked by a reporter in 1998 if he thought there would be hearings with "some of the principals," Graham said: "The depositions, I think, will determine whether or not we go forward with hearings. I think it\'s a very smart thing to do, to depose these people and find out what they\'ve got to say